In [ ]:
import os
import time

import requests
from bs4 import BeautifulSoup

from selenium.webdriver.common.keys import Keys

from tqdm.notebook import tqdm

from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
playlist = 0
URL = f"http://ocw.sharif.edu/course/id/{playlist}"
start_to_download = 0

# Get Course Data

In [ ]:
def get_soup(URL):
    #     driver = webdriver.Firefox(GeckoDriverManager().install())
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(URL)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    return soup

soup = get_soup(URL)

In [ ]:
title = soup.find('div', attrs={'id': 'd_desc'})
teacher = soup.find('div', attrs={'id': 'd_teacher'}).find_all('a')[1].get_text().strip()
print(teacher)

In [ ]:
path = 'D:\\Video_DL\\' + title.find('h1', attrs={'class':'-h2'}).get_text() + '-' + teacher
try:
    os.makedirs(path)
    print(path)
    print('The Folder is Created')
except:
    print(path)
    print('This Folder Is Exist')

# Download Courses

In [ ]:
download_urls = []

d_video = soup.find('div', attrs={'id': 'd_video'})
for tag in d_video.select('div.ng-hide'):
    tag.decompose()

seesion_list = d_video.find('div', attrs={'class': 'seesion_list'})
playlist_items = seesion_list.find_all('div', attrs={'class': 'ng-scope'})

In [ ]:
download_urls = []
with tqdm(total  =len(playlist_items)) as pbar:
    for index, item in enumerate(playlist_items):
        if (index+1) >= start_to_download:      
            a = item.find_all('a')        

            file_url = a[-1]['href']
            item_title = a[0]['ng-title'].strip()
            
#             if (item_title==''):
#                 item_title = item.find('div', attrs = {'class': 'ng-binding'}).get_text().strip()
            
            file_name = str(index+1) + '_' + item_title + '.mp4'
            file_name = "".join( x  if (x.isalnum() or x in "._-,، ") else '-' for x in file_name)
    #         <>:"/\|?*

            file_path = os.path.join(path, file_name)

            if len(file_path)>=260:
                file_path = file_path.split('.mp4')[0][:200]+'.mp4'

            if os.path.exists(file_path)==False:

                download_urls.append(file_url)
                try:
                    headers = {'Accept-Language': 'en-US,en;q=0.9,te;q=0.8'}
                    r = requests.get(file_url, allow_redirects=True, stream=True, headers=headers)  
                    print(r)
                    print(file_url)
                    if r != '<Response [404]>':
                        with open(file_path.strip(),"wb") as video: 
                            for chunk in r.iter_content(chunk_size=1024):         
                                if chunk: 
                                    video.write(chunk) 
                    else:
                        print(item_title, r)
                except:
                    print(f'Error: {item_title}')
                
        pbar.update(1)

# Download Practice

In [ ]:
d_practice = soup.find('div', attrs={'id': 'd_practice'})
for tag in d_practice.select('div.ng-hide'):
    tag.decompose()

seesion_list = d_practice.find('div', attrs={'class': 'seesion_list'})
playlist_items = seesion_list.find_all('div', attrs={'class': 'ng-scope'})

In [ ]:
download_urls = []
with tqdm(total  =len(playlist_items)) as pbar:
    for index, item in enumerate(playlist_items):
        a = item.find_all('a')        
      
        file_url = a[-1]['href']
        
        s = item.find('div', attrs={'class':'ng-binding'}).get_text()
        d=''
        for i in s.splitlines():
            d += i.strip()

        item_title = d.strip()

      
        file_name = str(index+1) + '_' + item_title + '.mp4'
        file_name = "".join( x  if (x.isalnum() or x in "._-,، ") else '-' for x in file_name)
#         <>:"/\|?*
        file_path = os.path.join(path, file_name)
        if os.path.exists(file_path)==False:
            
            download_urls.append(file_url)
            
            r = requests.get(file_url, stream = True)   
            with open(file_path,"wb") as video: 
                for chunk in r.iter_content(chunk_size=1024):         
                    if chunk: 
                        video.write(chunk) 
        pbar.update(1)